# Rich Context Knowledge Graph Visualization

In [3]:
!pip install pyvis
!pip install pandas

     |████████████████████████████████| 10.2MB 2.3MB/s eta 0:00:01    |███████████████▎                | 4.8MB 1.2MB/s eta 0:00:05
  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl
     |████████████████████████████████| 15.1MB 3.4MB/s eta 0:00:01


In [73]:
import json

filename = "tmp.jsonld"

with open(filename, "r") as f:
    jld_corpus = json.load(f)
    corpus = jld_corpus["@graph"]

IDS = []
publications = {}
datasets = {}
journals = {}


def get_id (id):
    global IDS
    return int(IDS.index(id))

    
# load the datasets

for elem in corpus:
    kind = elem["@type"]
    title = elem["dct:title"]["@value"]

    id = elem["@id"].split("#")[1]
    IDS.append(id)

    if kind == "Dataset":
        provider = elem["dct:publisher"]["@value"]

        view = {
            "id": id,
            "title": title,
            "provider": provider
        }

        datasets[id] = view


# load the journals

for elem in corpus:
    kind = elem["@type"]
    title = elem["dct:title"]["@value"]

    id = elem["@id"].split("#")[1]
    ids.append(id)

    if kind == "Journal":
        if "dct:identifier" in elem:
            issn = elem["dct:identifier"]["@value"]
        else:
            issn = ""

        view = {
            "id": id,
            "title": title,
            "issn": issn
        }

        journals[id] = view


# load the publications

for elem in corpus:
    kind = elem["@type"]
    title = elem["dct:title"]["@value"]

    id = elem["@id"].split("#")[1]
    ids.append(id)

    if kind == "ResearchPublication":
        dat_list = []
        c = elem["cito:citesAsDataSource"]

        if isinstance(c, dict):
            c = [c]
            
        for d in c:
            dat_id = d["@id"].split("#")[1]
            datasets[dat_id]["used"] = True
            dat_list.append(dat_id)

        if "dct:identifier" in elem:
            doi = elem["dct:identifier"]["@value"]

            if not doi.startswith("10."):
                doi = ""

        if "dct:publisher" in elem:
            jour_id = elem["dct:publisher"]["@id"].split("#")[1]
            journals[jour_id]["used"] = True
        else:
            journal = None

        view = {
            "id": id,
            "title": title,
            "doi": doi,
            "journal": jour_id,
            "datasets": dat_list
        }

        publications[id] = view

In [74]:
from pyvis.network import Network

g = Network(notebook=True, height="1000px", width="100%")
g.force_atlas_2based()

for d in datasets.values():
    if "used" in d:
        title = "{}<br/>{}".format(d["title"], d["provider"])
        g.add_node(get_id(d["id"]), label=d["title"], title=title, color="red")

for j in journals.values():
    if "used" in j:
        title = "{}<br/>{}".format(j["title"], j["issn"])
        g.add_node(get_id(j["id"]), label=j["title"], title=title, color="green")

for p in publications.values():
    title = "{}<br/>{}".format(p["title"], p["doi"])
    g.add_node(get_id(p["id"]), label=p["title"], title=title, color="blue")

    if p["journal"]:
        g.add_edge(get_id(p["id"]), get_id(p["journal"]), color="gray")

    for d in p["datasets"]:
        g.add_edge(get_id(p["id"]), get_id(d), color="gray")

g.show_buttons()
g.show("corpus.html")